<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/RS%C4%B0_LONG_S%C4%B0NYAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import time
import requests
from termcolor import colored
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

def calculate_rsi_wilders(data, column='close', period=14):
    delta = data[column].diff(1)
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)

    avg_gain = gain.ewm(alpha=1/period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False).mean()

    rs = avg_gain / avg_loss
    return 100.0 - (100.0 / (1.0 + rs))

def get_binance_data(pair):
    try:
        # 1. Veri Çekme
        url = f"https://api.binance.com/api/v3/klines?symbol={pair}&interval=1h&limit=30"
        response = requests.get(url, timeout=20)

        if response.status_code != 200:
            return (pair, None, f"HTTP Error {response.status_code}")

        data = response.json()

        # 2. Veri Kontrolü
        if len(data) < 25:
            return (pair, None, "Yetersiz veri")

        df = pd.DataFrame(data, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_asset_volume', 'trades',
            'taker_buy_base', 'taker_buy_quote', 'ignore'
        ])

        # 3. Numerik Dönüşüm
        numeric_cols = ['open', 'high', 'low', 'close']
        df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

        # 4. Geçersiz Veri Filtreleme
        if df['close'].isnull().any() or df['high'].isnull().any() or df['low'].isnull().any():
            return (pair, None, "Geçersiz fiyat verisi")

        # 5. Logaritmik Hesaplamalar
        df['logHigh'] = np.log(df['high'].replace(0, np.nan))
        df['logLow'] = np.log(df['low'].replace(0, np.nan))
        df['logClose'] = np.log(df['close'].replace(0, np.nan))

        # 6. RSI Hesaplama
        try:
            df['rsihigh'] = calculate_rsi_wilders(df, 'logHigh')
            df['rsilow'] = calculate_rsi_wilders(df, 'logLow')
            df['rsiclose'] = calculate_rsi_wilders(df, 'logClose')
        except Exception as e:
            return (pair, None, f"RSI hesaplama hatası: {str(e)}")

        # 7. Değişim Oranları
        df['rsiChangeclose'] = df['rsiclose'].diff()
        df['rsiChangelow'] = df['rsilow'].diff()

        # 8. Sinyal Tespiti (Orijinal Mantık)
        signals = []
        for i in range(-5, 0):
            try:
                current_time = datetime.fromtimestamp(df['timestamp'].iloc[i]/1000)
                close_change = df['rsiChangeclose'].iloc[i]
                low_change = df['rsiChangelow'].iloc[i]

                conditions = []
                if close_change > 20:
                    conditions.append(f"{current_time:%H:%M} CLOSE Δ+{close_change:.1f}>20")
                elif close_change > 10:
                    conditions.append(f"{current_time:%H:%M} CLOSE Δ+{close_change:.1f}>10")

                if low_change > 20:
                    conditions.append(f"{current_time:%H:%M} LOW Δ+{low_change:.1f}>20")

                if conditions:
                    signals.append(" | ".join(conditions))

            except Exception:
                continue

        return (pair, df['close'].iloc[-1], signals[-3:] if signals else None)

    except Exception as e:
        return (pair, None, f"Hata: {str(e)}")

if __name__ == "__main__":
    manual_pairs = [
    'BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'NEOUSDT', 'LTCUSDT', 'QTUMUSDT', 'ADAUSDT',
    'XRPUSDT', 'EOSUSDT', 'TUSDUSDT', 'IOTAUSDT', 'XLMUSDT', 'ONTUSDT', 'TRXUSDT',
    'ETCUSDT', 'ICXUSDT', 'NULSUSDT', 'VETUSDT', 'USDCUSDT', 'LINKUSDT', 'ONGUSDT',
    'HOTUSDT', 'ZILUSDT', 'ZRXUSDT', 'FETUSDT', 'BATUSDT', 'ZECUSDT', 'IOSTUSDT',
    'CELRUSDT', 'DASHUSDT', 'THETAUSDT', 'ENJUSDT', 'ATOMUSDT', 'TFUELUSDT', 'ONEUSDT',
    'ALGOUSDT', 'DOGEUSDT', 'DUSKUSDT', 'ANKRUSDT', 'WINUSDT', 'COSUSDT', 'MTLUSDT',
    'DENTUSDT', 'WANUSDT', 'FUNUSDT', 'CVCUSDT', 'CHZUSDT', 'BANDUSDT', 'XTZUSDT',
    'RVNUSDT', 'HBARUSDT', 'NKNUSDT', 'STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT',
    'RLCUSDT', 'CTXCUSDT', 'BCHUSDT', 'TROYUSDT', 'VITEUSDT', 'FTTUSDT', 'EURUSDT',
    'OGNUSDT', 'LSKUSDT', 'BNTUSDT', 'LTOUSDT', 'MBLUSDT', 'COTIUSDT', 'STPTUSDT',
    'DATAUSDT', 'SOLUSDT', 'CTSIUSDT', 'HIVEUSDT', 'CHRUSDT', 'ARDRUSDT', 'MDTUSDT',
    'STMXUSDT', 'KNCUSDT', 'LRCUSDT', 'COMPUSDT', 'SCUSDT', 'ZENUSDT', 'SNXUSDT',
    'VTHOUSDT', 'DGBUSDT', 'SXPUSDT', 'MKRUSDT', 'DCRUSDT', 'STORJUSDT', 'MANAUSDT',
    'YFIUSDT', 'BALUSDT', 'KMDUSDT', 'JSTUSDT', 'CRVUSDT', 'SANDUSDT', 'NMRUSDT',
    'DOTUSDT', 'LUNAUSDT', 'RSRUSDT', 'PAXGUSDT', 'TRBUSDT', 'SUSHIUSDT', 'KSMUSDT',
    'EGLDUSDT', 'DIAUSDT', 'RUNEUSDT', 'FIOUSDT', 'UMAUSDT', 'BELUSDT', 'WINGUSDT',
    'UNIUSDT', 'OXTUSDT', 'SUNUSDT', 'AVAXUSDT', 'FLMUSDT', 'UTKUSDT', 'XVSUSDT',
    'ALPHAUSDT', 'AAVEUSDT', 'NEARUSDT', 'FILUSDT', 'INJUSDT', 'AUDIOUSDT', 'CTKUSDT',
    'AXSUSDT', 'HARDUSDT', 'STRAXUSDT', 'ROSEUSDT', 'AVAUSDT', 'SKLUSDT', 'GRTUSDT',
    'JUVUSDT', 'PSGUSDT', '1INCHUSDT', 'OGUSDT', 'ATMUSDT', 'ASRUSDT', 'CELOUSDT',
    'RIFUSDT', 'TRUUSDT', 'CKBUSDT', 'TWTUSDT', 'FIROUSDT', 'LITUSDT', 'SFPUSDT',
    'DODOUSDT', 'CAKEUSDT', 'ACMUSDT', 'BADGERUSDT', 'FISUSDT', 'OMUSDT', 'PONDUSDT',
    'DEGOUSDT', 'ALICEUSDT', 'LINAUSDT', 'PERPUSDT', 'SUPERUSDT', 'CFXUSDT', 'TKOUSDT',
    'PUNDIXUSDT', 'TLMUSDT', 'BARUSDT', 'FORTHUSDT', 'BAKEUSDT', 'BURGERUSDT', 'SLPUSDT',
    'SHIBUSDT', 'ICPUSDT', 'ARUSDT', 'MASKUSDT', 'LPTUSDT', 'XVGUSDT', 'ATAUSDT',
    'GTCUSDT', 'ERNUSDT', 'PHAUSDT', 'MLNUSDT', 'DEXEUSDT', 'C98USDT', 'CLVUSDT',
    'QNTUSDT', 'FLOWUSDT', 'MINAUSDT', 'RAYUSDT', 'FARMUSDT', 'ALPACAUSDT', 'QUICKUSDT',
    'MBOXUSDT', 'REQUSDT', 'GHSTUSDT', 'WAXPUSDT', 'GNOUSDT', 'XECUSDT', 'ELFUSDT',
    'DYDXUSDT', 'IDEXUSDT', 'VIDTUSDT', 'USDPUSDT', 'GALAUSDT', 'ILVUSDT', 'YGGUSDT',
    'SYSUSDT', 'DFUSDT', 'FIDAUSDT', 'AGLDUSDT', 'RADUSDT', 'BETAUSDT', 'RAREUSDT',
    'LAZIOUSDT', 'CHESSUSDT', 'ADXUSDT', 'AUCTIONUSDT', 'BNXUSDT', 'MOVRUSDT', 'CITYUSDT',
    'ENSUSDT', 'QIUSDT', 'PORTOUSDT', 'POWRUSDT', 'JASMYUSDT', 'AMPUSDT', 'PYRUSDT',
    'ALCXUSDT', 'SANTOSUSDT', 'BICOUSDT', 'FLUXUSDT', 'FXSUSDT', 'VOXELUSDT', 'HIGHUSDT',
    'CVXUSDT', 'PEOPLEUSDT', 'SPELLUSDT', 'JOEUSDT', 'ACHUSDT', 'IMXUSDT', 'GLMRUSDT',
    'LOKAUSDT', 'SCRTUSDT', 'API3USDT', 'BTTCUSDT', 'ACAUSDT', 'XNOUSDT', 'WOOUSDT',
    'ALPINEUSDT', 'TUSDT', 'ASTRUSDT', 'GMTUSDT', 'KDAUSDT', 'APEUSDT', 'BSWUSDT',
    'BIFIUSDT', 'STEEMUSDT', 'NEXOUSDT', 'REIUSDT', 'LDOUSDT', 'OPUSDT', 'LEVERUSDT',
    'STGUSDT', 'LUNCUSDT', 'GMXUSDT', 'POLYXUSDT', 'APTUSDT', 'OSMOUSDT', 'HFTUSDT',
    'PHBUSDT', 'HOOKUSDT', 'MAGICUSDT', 'HIFIUSDT', 'RPLUSDT', 'PROSUSDT', 'GNSUSDT',
    'SYNUSDT', 'VIBUSDT', 'SSVUSDT', 'LQTYUSDT', 'AMBUSDT', 'USTCUSDT', 'GASUSDT',
    'GLMUSDT', 'PROMUSDT', 'QKCUSDT', 'UFTUSDT', 'IDUSDT', 'ARBUSDT', 'RDNTUSDT',
    'WBTCUSDT', 'EDUUSDT', 'SUIUSDT', 'AERGOUSDT', 'PEPEUSDT', 'FLOKIUSDT', 'ASTUSDT',
    'SNTUSDT', 'COMBOUSDT', 'MAVUSDT', 'PENDLEUSDT', 'ARKMUSDT', 'WBETHUSDT', 'WLDUSDT',
    'FDUSDUSDT', 'SEIUSDT', 'CYBERUSDT', 'ARKUSDT', 'CREAMUSDT', 'IQUSDT', 'NTRNUSDT',
    'TIAUSDT', 'MEMEUSDT', 'ORDIUSDT', 'BEAMXUSDT', 'PIVXUSDT', 'VICUSDT', 'BLURUSDT',
    'VANRYUSDT', 'AEURUSDT', 'JTOUSDT', '1000SATSUSDT', 'BONKUSDT', 'ACEUSDT', 'NFPUSDT',
    'AIUSDT', 'XAIUSDT', 'MANTAUSDT', 'ALTUSDT', 'JUPUSDT', 'PYTHUSDT', 'RONINUSDT',
    'DYMUSDT', 'PIXELUSDT', 'STRKUSDT', 'PORTALUSDT', 'PDAUSDT', 'AXLUSDT', 'WIFUSDT',
    'METISUSDT', 'AEVOUSDT', 'BOMEUSDT', 'ETHFIUSDT', 'ENAUSDT', 'WUSDT', 'TNSRUSDT',
    'SAGAUSDT', 'TAOUSDT', 'OMNIUSDT', 'REZUSDT', 'BBUSDT', 'NOTUSDT', 'IOUSDT', 'ZKUSDT',
    'LISTAUSDT', 'ZROUSDT', 'GUSDT', 'BANANAUSDT', 'RENDERUSDT', 'TONUSDT', 'DOGSUSDT',
    'EURIUSDT', 'SLFUSDT', 'POLUSDT', 'NEIROUSDT', 'TURBOUSDT', '1MBABYDOGEUSDT',
    'CATIUSDT', 'HMSTRUSDT', 'EIGENUSDT', 'SCRUSDT', 'BNSOLUSDT', 'LUMIAUSDT', 'KAIAUSDT',
    'COWUSDT', 'CETUSUSDT', 'PNUTUSDT', 'ACTUSDT', 'USUALUSDT', 'THEUSDT', 'ACXUSDT',
    'ORCAUSDT', 'MOVEUSDT', 'MEUSDT', 'VELODROMEUSDT', 'VANAUSDT', '1000CATUSDT',
    'PENGUUSDT', 'BIOUSDT', 'DUSDT', 'AIXBTUSDT', 'CGPTUSDT', 'COOKIEUSDT', 'SUSDT',
    'SOLVUSDT', 'TRUMPUSDT', 'ANIMEUSDT'
]

# Aktif çift filtreleme (BCC, VEN gibi eski çiftleri otomatik temizle)
active_pairs = [p for p in manual_pairs if not any(x in p for x in ['BCC','VEN','BCHABC','BCHSV','USDS','BKRW'])]

start_time = time.time()
print(colored(f"🚀 Toplam {len(active_pairs)} aktif çift taranıyor...", "cyan", attrs=['bold']))

found = []
error_count = 0
processed = 0

with ThreadPoolExecutor(max_workers=3) as executor:
        futures = {executor.submit(get_binance_data, pair): pair for pair in active_pairs}

        for future in as_completed(futures):
            processed += 1
            pair = futures[future]

            # İlerleme çubuğu
            if processed % 5 == 0:
                progress = processed/len(active_pairs)*100
                print(colored(f"▰{'▰'*int(progress//5)}{'▱'*int(20-(progress//5))} {processed}/{len(active_pairs)} ({progress:.1f}%)", "yellow"))

            try:
                result = future.result()
                if result[1] is not None and result[2] is not None:
                    found.append(result)
                elif result[2]:
                    print(colored(f"⚠️ {result[0]}: {result[2]}", "red"))
                    error_count +=1
            except Exception as e:
                print(colored(f"❌ {pair} işlenirken kritik hata: {str(e)}", "red"))
                error_count +=1

            time.sleep(0.3)

    # Sonuçların Gösterimi
print(colored("\n" + "═"*50, "blue"))
print(colored("🔍 SONUÇLAR", "green", attrs=['bold']))
print(colored(f"• Sinyal Bulunan: {len(found)}", "cyan"))
print(colored(f"• Hatalı Çiftler: {error_count}", "red"))
print(colored(f"• Başarı Oranı: {(len(found)/(len(active_pairs)-error_count)*100 if (len(active_pairs)-error_count)!=0 else 0):.1f}%", "yellow"))
print(colored("═"*50 + "\n", "blue"))

if found:
        found.sort(key=lambda x: x[1], reverse=True)

        for coin in found:
            price = colored(f"${coin[1]:,.2f}", "yellow")
            header = colored(f"🎯 {coin[0]:<12}", "magenta") + colored("│ ", "white") + price
            print(header)

            for signal in coin[2]:
                parts = []
                for part in signal.split("|"):
                    if "CLOSE Δ" in part:
                        parts.append(colored(part.strip(), "green"))
                    elif "LOW Δ" in part:
                        parts.append(colored(part.strip(), "cyan"))
                print(colored("   ├─ ", "white") + " | ".join(parts))
            print(colored("   └────────────────────────", "white"))
else:
        print(colored("\n❌ Hiç sinyal bulunamadı", "red", attrs=['bold']))

print(colored(f"\n⏱️ Toplam süre: {time.time()-start_time:.1f} saniye", "yellow", attrs=['bold']))

🚀 Toplam 394 aktif çift taranıyor...
▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 5/394 (1.3%)
▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 10/394 (2.5%)
▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 15/394 (3.8%)
▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 20/394 (5.1%)
▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 25/394 (6.3%)
▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 30/394 (7.6%)
▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 35/394 (8.9%)
▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 40/394 (10.2%)
▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 45/394 (11.4%)
▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 50/394 (12.7%)
▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 55/394 (14.0%)
▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 60/394 (15.2%)
▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 65/394 (16.5%)
▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 70/394 (17.8%)
▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 75/394 (19.0%)
▰▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 80/394 (20.3%)
▰▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 85/394 (21.6%)
▰▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 90/394 (22.8%)
▰▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 95/394 (24.1%)
▰▰▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 100/394 (25.4%)
▰▰▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 105/394 (26.6%)
▰▰▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 110/394 (27.9%)
▰▰▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 115/394 (29.2%)
▰▰▰▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 120/394 (30.5%)
▰▰▰▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 125/394 (31.7%)
▰▰▰▰▰▰▰▱▱▱▱▱▱▱▱▱▱▱▱▱▱ 130/394 (33.0%)
▰▰